# 参考答案

In [1]:
import pandas as pd
import numpy as np

## 第2章：练习一

### (a)

In [2]:
df = pd.read_csv('data/UFO.csv')
df.rename(columns={'duration (seconds)':'duration'},inplace=True)
df['duration'].astype('float')
df.head()

,datetime,shape,duration,latitude,longitude
0,10/10/1949 20:30,cylinder,2700.0,29.883056,-97.941111
1,10/10/1949 21:00,light,7200.0,29.384210,-98.581082
2,10/10/1955 17:00,circle,20.0,53.200000,-2.916667
3,10/10/1956 21:00,circle,20.0,28.978333,-96.645833
4,10/10/1960 20:00,light,900.0,21.418056,-157.803611


In [3]:
df.query('duration > 60')['shape'].value_counts().index[0]

'light'

### (b)

In [4]:
bins_long = np.linspace(-180,180,13).tolist()
bins_la = np.linspace(-90,90,11).tolist()
cuts_long = pd.cut(df['longitude'],bins=bins_long)
df['cuts_long'] = cuts_long
cuts_la = pd.cut(df['latitude'],bins=bins_la)
df['cuts_la'] = cuts_la
df.head()

,datetime,shape,duration,latitude,longitude,cuts_long,cuts_la
0,10/10/1949 20:30,cylinder,2700.0,29.883056,-97.941111,"(-120.0, -90.0]","(18.0, 36.0]"
1,10/10/1949 21:00,light,7200.0,29.384210,-98.581082,"(-120.0, -90.0]","(18.0, 36.0]"
2,10/10/1955 17:00,circle,20.0,53.200000,-2.916667,"(-30.0, 0.0]","(36.0, 54.0]"
3,10/10/1956 21:00,circle,20.0,28.978333,-96.645833,"(-120.0, -90.0]","(18.0, 36.0]"
4,10/10/1960 20:00,light,900.0,21.418056,-157.803611,"(-180.0, -150.0]","(18.0, 36.0]"


In [5]:
df.set_index(['cuts_long','cuts_la']).index.value_counts().index[0]

(Interval(-90.0, -60.0, closed='right'), Interval(36.0, 54.0, closed='right'))

## 第2章：练习二

### (a)

In [6]:
df = pd.read_csv('data/Pokemon.csv')
df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [7]:
df['Type 2'].count()/df.shape[0]

0.5175

### (b)

In [8]:
df.query('Total >= 580')['Legendary'].value_counts(normalize=True)

True     0.575221
False    0.424779
Name: Legendary, dtype: float64

### (c) 分别为Maga形态的路卡利欧、修缮老头、怪力

In [9]:
df[df['Type 1']=='Fighting'].sort_values(by='Attack',ascending=False).iloc[:3]

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
498,448,LucarioMega Lucario,Fighting,Steel,625,70,145,88,140,70,112,4,False
594,534,Conkeldurr,Fighting,NaN,505,105,140,95,55,65,45,5,False
74,68,Machamp,Fighting,NaN,505,90,130,80,65,85,55,1,False


### (d) 钢系的极差均值最大

In [10]:
df['range'] = df.iloc[:,5:11].max(axis=1)-df.iloc[:,5:11].min(axis=1)
attribute = df[['Type 1','range']].set_index('Type 1')
max_range = 0
result = ''
for i in attribute.index.unique():
    temp = attribute.loc[i,:].mean()
    if temp.values[0] > max_range:
        max_range = temp.values[0]
        result = i
result

'Steel'

### (e) 超能系占比最高，但普通系均值最高？？？（创世神BUG）

In [11]:
df.query('Legendary == True')['Type 1'].value_counts(normalize=True).index[0]

'Psychic'

In [12]:
attribute = df.query('Legendary == True')[['Type 1','Total']].set_index('Type 1')
max_value = 0
result = ''
for i in attribute.index.unique()[:-1]:
    temp = attribute.loc[i,:].mean()
    if temp[0] > max_value:
        max_value = temp[0]
        result = i
result

'Normal'

## 第3章：练习一

### (a) 极差为17561

In [13]:
df = pd.read_csv('data/Diamonds.csv')
df.head()

,carat,color,depth,price
0,0.23,E,61.5,326
1,0.21,E,59.8,326
2,0.23,E,56.9,327
3,0.29,I,62.4,334
4,0.31,J,63.3,335


In [14]:
df_r = df.query('carat>1')['price']
df_r.max()-df_r.min()

17561

### (b) 0-0.2分位数区间最多的为‘E’，其余区间都为‘G’

In [15]:
bins = df['depth'].quantile(np.linspace(0,1,6)).tolist()
cuts = pd.cut(df['depth'],bins=bins) #可选label添加自定义标签
df['cuts'] = cuts
df.head()

,carat,color,depth,price,cuts
0,0.23,E,61.5,326,"(60.8, 61.6]"
1,0.21,E,59.8,326,"(43.0, 60.8]"
2,0.23,E,56.9,327,"(43.0, 60.8]"
3,0.29,I,62.4,334,"(62.1, 62.7]"
4,0.31,J,63.3,335,"(62.7, 79.0]"


In [16]:
color_result = df.groupby('cuts')['color'].describe()
color_result

,count,unique,top,freq
cuts,,,,
"(43.0, 60.8]",11294,7,E,2259
"(60.8, 61.6]",11831,7,G,2593
"(61.6, 62.1]",10403,7,G,2247
"(62.1, 62.7]",10137,7,G,2193
"(62.7, 79.0]",10273,7,G,2000


### 前三个分位数区间不满足条件，后两个区间中数量最多的颜色的确是均重价格中最贵的

In [17]:
df['均重价格']=df['price']/df['carat']
color_result['top'] == [i[1] for i in df.groupby(['cuts'
                                ,'color'])['均重价格'].mean().groupby(['cuts']).idxmax().values]

cuts
(43.0, 60.8]    False
(60.8, 61.6]    False
(61.6, 62.1]    False
(62.1, 62.7]     True
(62.7, 79.0]     True
Name: top, dtype: bool

### (c) 结果见下：

In [18]:
df = df.drop(columns='均重价格')
cuts = pd.cut(df['carat'],bins=[0,0.5,1,1.5,2,np.inf]) #可选label添加自定义标签
df['cuts'] = cuts
df.head()

,carat,color,depth,price,cuts
0,0.23,E,61.5,326,"(0.0, 0.5]"
1,0.21,E,59.8,326,"(0.0, 0.5]"
2,0.23,E,56.9,327,"(0.0, 0.5]"
3,0.29,I,62.4,334,"(0.0, 0.5]"
4,0.31,J,63.3,335,"(0.0, 0.5]"


In [19]:
def f(nums):
    if not nums:        
        return 0
    res = 1                            
    cur_len = 1                        
    for i in range(1, len(nums)):      
        if nums[i-1] < nums[i]:        
            cur_len += 1                
            res = max(cur_len, res)     
        else:                       
            cur_len = 1                 
    return res

In [20]:
for name,group in df.groupby('cuts'):
    group = group.sort_values(by='depth')
    s = group['price']
    print(name,f(s.tolist()))

(0.0, 0.5] 8
(0.5, 1.0] 8
(1.0, 1.5] 7
(1.5, 2.0] 11
(2.0, inf] 7


### (d) 计算结果如下：

In [21]:
for name,group in df[['carat','price','color']].groupby('color'):
    L1 = np.array([np.ones(group.shape[0]),group['carat']]).reshape(2,group.shape[0])
    L2 = group['price']
    result = (np.linalg.inv(L1.dot(L1.T)).dot(L1)).dot(L2).reshape(2,1)
    print('当颜色为%s时，截距项为：%f，回归系数为：%f'%(name,result[0],result[1]))

当颜色为D时，截距项为：-2361.017152，回归系数为：8408.353126
当颜色为E时，截距项为：-2381.049600，回归系数为：8296.212783
当颜色为F时，截距项为：-2665.806191，回归系数为：8676.658344
当颜色为G时，截距项为：-2575.527643，回归系数为：8525.345779
当颜色为H时，截距项为：-2460.418046，回归系数为：7619.098320
当颜色为I时，截距项为：-2878.150356，回归系数为：7761.041169
当颜色为J时，截距项为：-2920.603337，回归系数为：7094.192092


## 第3章：练习二

### (a)

In [22]:
df = pd.read_csv('data/Drugs.csv')
df.head()

,YYYY,State,COUNTY,SubstanceName,DrugReports
0,2010,VA,ACCOMACK,Propoxyphene,1
1,2010,OH,ADAMS,Morphine,9
2,2010,PA,ADAMS,Methadone,2
3,2010,VA,ALEXANDRIA CITY,Heroin,5
4,2010,PA,ALLEGHENY,Hydromorphone,5


In [23]:
idx=pd.IndexSlice
for i in range(2010,2018):
    county = (df.groupby(['COUNTY','YYYY']).sum().loc[idx[:,i],:].idxmax()[0][0])
    state = df.query('COUNTY == "%s"'%county)['State'].iloc[0]
    state_true = df.groupby(['State','YYYY']).sum().loc[idx[:,i],:].idxmax()[0][0]
    if state==state_true:
        print('在%d年，%s县的报告数最多，它所属的州%s也是报告数最多的'%(i,county,state))
    else:
        print('在%d年，%s县的报告数最多，但它所属的州%s不是报告数最多的，%s州报告数最多'%(i,county,state,state_true))

在2010年，PHILADELPHIA县的报告数最多，它所属的州PA也是报告数最多的
在2011年，PHILADELPHIA县的报告数最多，但它所属的州PA不是报告数最多的，OH州报告数最多
在2012年，PHILADELPHIA县的报告数最多，但它所属的州PA不是报告数最多的，OH州报告数最多
在2013年，PHILADELPHIA县的报告数最多，但它所属的州PA不是报告数最多的，OH州报告数最多
在2014年，PHILADELPHIA县的报告数最多，但它所属的州PA不是报告数最多的，OH州报告数最多
在2015年，PHILADELPHIA县的报告数最多，但它所属的州PA不是报告数最多的，OH州报告数最多
在2016年，HAMILTON县的报告数最多，它所属的州OH也是报告数最多的
在2017年，HAMILTON县的报告数最多，它所属的州OH也是报告数最多的


### (b) OH州增加最多，Heroin是增量最大的，但增幅最大的是Acetyl fentanyl

In [24]:
df_b = df[(df['YYYY'].isin([2014,2015]))&(df['SubstanceName']=='Heroin')]
df_add = df_b.groupby(['YYYY','State']).sum()
(df_add.loc[2015]-df_add.loc[2014]).idxmax()

DrugReports    OH
dtype: object

In [25]:
df_b = df[(df['YYYY'].isin([2014,2015]))&(df['State']=='OH')]
df_add = df_b.groupby(['YYYY','SubstanceName']).sum()
display((df_add.loc[2015]-df_add.loc[2014]).idxmax()) #这里利用了索引对齐的特点
display((df_add.loc[2015]/df_add.loc[2014]).idxmax())

DrugReports    Heroin
dtype: object

DrugReports    Acetyl fentanyl
dtype: object

## 第4章：练习一

### (a)

In [26]:
df = pd.read_csv('data/Drugs.csv',index_col=['State','COUNTY']).sort_index()
df.head()

YYYY  SubstanceName  DrugReports
State COUNTY                                  
KY    ADAIR   2010      Methadone            1
      ADAIR   2010    Hydrocodone            6
      ADAIR   2011      Oxycodone            4
      ADAIR   2011  Buprenorphine            3
      ADAIR   2011       Morphine            2

In [27]:
result = pd.pivot_table(df,index=['State','COUNTY','SubstanceName']
                 ,columns='YYYY'
                 ,values='DrugReports',fill_value='-').reset_index()
result.columns.name=''
result.head()

,State,COUNTY,SubstanceName,2010,2011,2012,2013,2014,2015,2016,2017
0,KY,ADAIR,Buprenorphine,-,3,5,4,27,5,7,10
1,KY,ADAIR,Codeine,-,-,1,-,-,-,-,1
2,KY,ADAIR,Fentanyl,-,-,1,-,-,-,-,-
3,KY,ADAIR,Heroin,-,-,1,2,-,1,-,2
4,KY,ADAIR,Hydrocodone,6,9,10,10,9,7,11,3


### (b)

In [28]:
result_melted = result.melt(id_vars=result.columns[:3],value_vars=result.columns[-8:]
                ,var_name='YYYY',value_name='DrugReports').query('DrugReports != "-"')
result2 = result_melted.sort_values(by=['State','COUNTY','YYYY'
                                    ,'SubstanceName']).reset_index().drop(columns='index')
cols = list(result2.columns)
a, b = cols.index('SubstanceName'), cols.index('YYYY')
cols[b], cols[a] = cols[a], cols[b]
result2 = result2[cols]
result2[['DrugReports','YYYY']] = result2[['DrugReports','YYYY']].astype('int')
result2.head()

,State,COUNTY,YYYY,SubstanceName,DrugReports
0,KY,ADAIR,2010,Hydrocodone,6
1,KY,ADAIR,2010,Methadone,1
2,KY,ADAIR,2011,Buprenorphine,3
3,KY,ADAIR,2011,Hydrocodone,9
4,KY,ADAIR,2011,Morphine,2


In [29]:
df_tidy = df.reset_index().sort_values(by=result2.columns[:4].tolist()).reset_index().drop(columns='index')
df_tidy.head()

,State,COUNTY,YYYY,SubstanceName,DrugReports
0,KY,ADAIR,2010,Hydrocodone,6
1,KY,ADAIR,2010,Methadone,1
2,KY,ADAIR,2011,Buprenorphine,3
3,KY,ADAIR,2011,Hydrocodone,9
4,KY,ADAIR,2011,Morphine,2


In [30]:
df_tidy.equals(result2)

True

## 第4章：练习二

### (a)

In [31]:
df = pd.read_csv('data/Earthquake.csv')
df.index.name = 'index'
df.head()

,日期,时间,维度,经度,方向,距离,深度,烈度
index,,,,,,,,
0,2003.05.20,12:17:44 AM,39.04,40.38,west,0.1,10.0,0.0
1,2007.08.01,12:03:08 AM,40.79,30.09,west,0.1,5.2,4.0
2,1978.05.07,12:41:37 AM,38.58,27.61,south_west,0.1,0.0,0.0
3,1997.03.22,12:31:45 AM,39.47,36.44,south_west,0.1,10.0,0.0
4,2000.04.02,12:57:38 AM,40.80,30.24,south_west,0.1,7.0,0.0


In [32]:
df_pivoted = pd.pivot_table(df,index=df.index,columns='方向',values=df.columns[-3:],
                            fill_value='-').stack(level=0)
df_final = df[df.columns[:4]].join(df_pivoted.reset_index(level=1)).rename(columns={'level_1':'地震参数'})
df_final.head()

,日期,时间,维度,经度,地震参数,east,north,north_east,north_west,south,south_east,south_west,west
index,,,,,,,,,,,,,
0,2003.05.20,12:17:44 AM,39.04,40.38,深度,-,-,-,-,-,-,-,10
0,2003.05.20,12:17:44 AM,39.04,40.38,烈度,-,-,-,-,-,-,-,0
0,2003.05.20,12:17:44 AM,39.04,40.38,距离,-,-,-,-,-,-,-,0.1
1,2007.08.01,12:03:08 AM,40.79,30.09,深度,-,-,-,-,-,-,-,5.2
1,2007.08.01,12:03:08 AM,40.79,30.09,烈度,-,-,-,-,-,-,-,4


### (b)

In [33]:
df_melted = df_final.reset_index().melt(id_vars=df_final.reset_index().columns[:6]
                ,value_vars=df_final.columns[-8:],var_name='方向',value_name='数量')
df_melted = df_melted[df_melted['数量']!='-']
df_pivoted = df_melted.pivot(index='index',columns='地震参数',values='数量').astype('float64')
result = (df_melted.iloc[:,[0,1,2,3,4,6]][~df_melted.duplicated('index')].sort_values(
            by='index').join(df_pivoted,on='index')).set_index('index').reindex(range(df.shape[0]))
result = result.iloc[:,:5].join(result.iloc[:,[7,5]]).join(result.iloc[:,6])
result.head()

,日期,时间,维度,经度,方向,距离,深度,烈度
index,,,,,,,,
0,2003.05.20,12:17:44 AM,39.04,40.38,west,0.1,10.0,0.0
1,2007.08.01,12:03:08 AM,40.79,30.09,west,0.1,5.2,4.0
2,1978.05.07,12:41:37 AM,38.58,27.61,south_west,0.1,0.0,0.0
3,1997.03.22,12:31:45 AM,39.47,36.44,south_west,0.1,10.0,0.0
4,2000.04.02,12:57:38 AM,40.80,30.24,south_west,0.1,7.0,0.0


In [34]:
df.equals(result)

True

## 第5章：练习一

### (a)

In [35]:
df1 = pd.read_csv('data/Employee1.csv')
df2 = pd.read_csv('data/Employee2.csv')
df1.head()

,Company,Name,Age,Height,Weight,Salary
0,A,a1,47,188,63.7,25819
1,A,a3,39,172,55.9,21983
2,A,a4,43,158,62.5,21755
3,A,a6,42,182,76.9,17354
4,A,a7,49,171,94.6,6177


In [36]:
df2.head()

,Company,Name,Age,Height,Weight,Salary
0,A,a1,30,156,91.2,28133
1,A,a2,50,190,83.4,6673
2,A,a3,34,168,96.6,16503
3,A,a5,51,176,97.2,23294
4,A,a6,37,183,93.2,19256


In [37]:
L = list(set(df1['Name']).intersection(set(df2['Name'])))
L

['a3',
 'b1',
 'b7',
 'd5',
 'd10',
 'e10',
 'b3',
 'c10',
 'a6',
 'a1',
 'e8',
 'c12',
 'e11',
 'c13',
 'b15',
 'c3']

### (b)

In [38]:
df_b1 = df1[~df1['Name'].isin(L)]
df_b2 = df2[~df2['Name'].isin(L)]
df_b = pd.concat([df_b1,df_b2]).set_index('Name')
df_b.head()

,Company,Age,Height,Weight,Salary
Name,,,,,
a4,A,43,158,62.5,21755
a7,A,49,171,94.6,6177
a8,A,51,168,89.5,3246
a9,A,36,186,62.8,3569
a13,A,58,190,75.9,21854


### (c)

In [39]:
df1 = pd.read_csv('data/Employee1.csv')
df2 = pd.read_csv('data/Employee2.csv')
df1['重复'] = ['Y_1' if df1.loc[i,'Name'] in L else 'N' for i in range(df1.shape[0])]
df2['重复'] = ['Y_2' if df2.loc[i,'Name'] in L else 'N' for i in range(df2.shape[0])]
df1 = df1.set_index(['Name','重复'])
df2 = df2.set_index(['Name','重复'])
df_c = pd.concat([df1,df2])
result = pd.DataFrame({'Company':[],'Name':[],'Age':[],'Height':[],'Weight':[],'Salary':[]})
group = df_c.groupby(['Company','重复'])
for i in L:
    first = group.get_group((i[0].upper(),'Y_1')).reset_index(level=1).loc[i,:][-4:]
    second = group.get_group((i[0].upper(),'Y_2')).reset_index(level=1).loc[i,:][-4:]
    mean = group.get_group((i[0].upper(),'N')).reset_index(level=1).mean()
    final = [i[0].upper(),i]
    for j in range(4):
        final.append(first[j] if abs(first[j]-mean[j])<abs(second[j]-mean[j]) else second[j])
    result = pd.concat([result,pd.DataFrame({result.columns.tolist()[k]:[final[k]] for k in range(6)})])
result = pd.concat([result.set_index('Name'),df_b])
for i in list('abcde'):
    for j in range(1,17):
        item = i+str(j)
        if item not in result.index:
            result = pd.concat([result,pd.DataFrame({'Company':[i.upper()],'Name':[item]
                 ,'Age':[np.nan],'Height':[np.nan],'Weight':[np.nan],'Salary':[np.nan]}).set_index('Name')])
result['Number'] = [int(i[1:]) for i in result.index]
result.reset_index().drop(columns='Name').set_index(['Company','Number']).sort_index()

Age  Height  Weight   Salary
Company Number                               
A       1       47.0   188.0    91.2  25819.0
        2       50.0   190.0    83.4   6673.0
        3       39.0   172.0    96.6  16503.0
        4       43.0   158.0    62.5  21755.0
        5       51.0   176.0    97.2  23294.0
...              ...     ...     ...      ...
E       12      54.0   157.0    79.4  18490.0
        13      57.0   180.0    54.8  26837.0
        14      39.0   163.0    83.0  20554.0
        15       NaN     NaN     NaN      NaN
        16       NaN     NaN     NaN      NaN

[80 rows x 4 columns]

## 第5章：练习二

### (a)

In [40]:
df1 = pd.read_csv('data/Course1.csv')
df2 = pd.read_csv('data/Course2.csv')
df_a11,df_a12,df_a21,df_a22 =0,0,0,0
df_a11= df1.query('课程类别 in ["学科基础课","专业必修课","专业选修课"]')
df_a12= df1.query('课程类别 not in ["学科基础课","专业必修课","专业选修课"]')
df_a21= df2.query('课程类别 in ["学科基础课","专业必修课","专业选修课"]')
df_a22= df2.query('课程类别 not in ["学科基础课","专业必修课","专业选修课"]')
df_a11.head()

,课程名字,课程类别,学分,分数
1,云计算应用与开发,专业选修课,3,96.0
2,社会计算,专业选修课,3,78.0
3,深度学习,专业选修课,3,75.0
4,人工智能导论,专业必修课,3,84.0
6,数据结构与算法,学科基础课,5,82.0


### (b)

In [41]:
special = pd.concat([df_a11,df_a21])
common = pd.concat([df_a12,df_a22])
special.query('课程类别 not in ["学科基础课","专业必修课","专业选修课"]')

,课程名字,课程类别,学分,分数


In [42]:
common.query('课程类别 in ["学科基础课","专业必修课","专业选修课"]')

,课程名字,课程类别,学分,分数


### (c)

In [43]:
df = pd.concat([df1,df2])
special2 = df.query('课程类别 in ["学科基础课","专业必修课","专业选修课"]')
common2 = df.query('课程类别 not in ["学科基础课","专业必修课","专业选修课"]')
(special.equals(special2),common.equals(common2))

(True, True)

### (d)

In [44]:
df['分数'] = df.groupby('课程类别').transform(lambda x: x.fillna(x.mean()))['分数']
df.isnull().all()

课程名字    False
课程类别    False
学分      False
分数      False
dtype: bool

In [45]:
special3 = df.query('课程类别 in ["学科基础课","专业必修课","专业选修课"]')
common3 = df.query('课程类别 not in ["学科基础课","专业必修课","专业选修课"]')
common3.head()

,课程名字,课程类别,学分,分数
0,思想道德修养与法律基础,思政类,3,89.0
5,中国近代史纲要,思政类,3,97.0
8,网球（初）,体育类,1,81.0
10,极端性气候与陆地生态系统,公共任意选修类,2,78.0
13,游泳（初）,体育类,1,75.0
